In [4]:
from re import search

from matplotlib.pyplot import spring

from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter, deque
import re
from tqdm import tqdm
import itertools
from datetime import datetime

year = datetime.now().year
day = datetime.now().day

if not os.path.exists("inputs"):
    os.mkdir("inputs")

if not os.path.exists(f"inputs/day_{day}_input.txt"):
    download_input(year, day)

aoc, lines, G, R, C = read_input(day, test=False)

lines[:5]=['[#.#.] (0,2) (0,1,3) {28,20,8,20}', '[#.####.##.] (2,9) (0,1,3,4,5,6,8,9) (7,9) (0,4,9) (4,5,6) (1,2,4,7,8,9) (2,7,9) (1,2,3,4,5) (0,1,3,4,5,7,9) (0,7) (1,3,4,5,7) (1,3) {27,64,209,46,62,34,10,220,27,233}', '[..#...#.] (0,1,4,5,6,7) (0,2,3) (0,1,2,3,5,6,7) (0,1,2,3,6) (1,4) (0,1,2,5,6,7) {215,215,28,19,198,204,204,204}', '[......#.] (0,3) (0,1,2,4,5,6) (0,3,4,6) (1,2,4) (0,1,3,4,6) (0,1,2) (1,2,3,5) (2,5,7) (0,2) (0,1,5) {100,82,74,65,58,68,52,13}', '[#.##...] (3,6) (2,4,5) (3) (1,2,3) (0,1,2,3) (1,4,5) (0,2,6) {20,30,39,30,20,20,16}']
R=168, C=33


In [14]:
import heapq

aoc, lines, G, R, C = read_input(day, test=False)

total = 0

for machine in tqdm(lines):
    lights, controls = machine.split("]")
    lights = lights[1:]
    buttons, joltages = controls.split("{")
    buttons = [list(map(int,x[1:-1].split(","))) for x in buttons.split()]

    target_lights = tuple(True if lights[i] == '#' else False for i in range(len(lights)))

    Q = []

    visited = set()

    heapq.heappush(Q, (0, tuple(False for i in range(len(lights)))))
    while Q:
        num_presses, lights = heapq.heappop(Q)
        if  target_lights == lights:
            total += num_presses
            break
        else:
            for b in buttons:
                new_lights = list(lights)  # or list(lights)
                for s in b:
                    new_lights[s] = not new_lights[s]

                new_lights = tuple(new_lights)
                if new_lights not in visited:
                    visited.add(new_lights)
                    heapq.heappush(Q, (num_presses+1, new_lights))
print(total)


lines[:5]=['[#.#.] (0,2) (0,1,3) {28,20,8,20}', '[#.####.##.] (2,9) (0,1,3,4,5,6,8,9) (7,9) (0,4,9) (4,5,6) (1,2,4,7,8,9) (2,7,9) (1,2,3,4,5) (0,1,3,4,5,7,9) (0,7) (1,3,4,5,7) (1,3) {27,64,209,46,62,34,10,220,27,233}', '[..#...#.] (0,1,4,5,6,7) (0,2,3) (0,1,2,3,5,6,7) (0,1,2,3,6) (1,4) (0,1,2,5,6,7) {215,215,28,19,198,204,204,204}', '[......#.] (0,3) (0,1,2,4,5,6) (0,3,4,6) (1,2,4) (0,1,3,4,6) (0,1,2) (1,2,3,5) (2,5,7) (0,2) (0,1,5) {100,82,74,65,58,68,52,13}', '[#.##...] (3,6) (2,4,5) (3) (1,2,3) (0,1,2,3) (1,4,5) (0,2,6) {20,30,39,30,20,20,16}']
R=168, C=33


100%|██████████| 168/168 [00:00<00:00, 2966.14it/s]

438


In [53]:
# Using heap for optimal was way too slow. try linear programming approach using z3.
from z3 import Sum, sat, Optimize, Int

aoc, lines, G, R, C = read_input(day, test=False)

total = 0

for machine in lines:
    lights, controls = machine.split("]")
    lights = lights[1:]
    buttons, joltages = controls.split("{")
    buttons = [list(map(int,x[1:-1].split(","))) for x in buttons.split()]
    joltages = tuple(map(int,joltages[:-1].split(",")))

    x = []
    # Refact buttons into a vector.
    vec_buttons = []
    for i, b in enumerate(buttons):
        this_vec = [0]*len(joltages)
        for s in b:
            this_vec[s] += 1
        vec_buttons.append(this_vec)
        var_name = f'b{i}'
        x.append(Int(var_name))
    optimizer = Optimize()

    # Constraints: non-negative integers
    for var in x:
        optimizer.add(var >= 0)

    # Sum constraints for each dimension
    for j in range(len(joltages)):
        optimizer.add(Sum([x[i] * vec_buttons[i][j] for i in range(len(buttons))]) == joltages[j])
    optimizer.minimize(Sum(x))

    if optimizer.check() == sat:
        model = optimizer.model()
        solution = [model[var].as_long() for var in x]
        total += sum(solution)
    else:
        assert False

# 16901 too high
print(total)

lines[:5]=['[#.#.] (0,2) (0,1,3) {28,20,8,20}', '[#.####.##.] (2,9) (0,1,3,4,5,6,8,9) (7,9) (0,4,9) (4,5,6) (1,2,4,7,8,9) (2,7,9) (1,2,3,4,5) (0,1,3,4,5,7,9) (0,7) (1,3,4,5,7) (1,3) {27,64,209,46,62,34,10,220,27,233}', '[..#...#.] (0,1,4,5,6,7) (0,2,3) (0,1,2,3,5,6,7) (0,1,2,3,6) (1,4) (0,1,2,5,6,7) {215,215,28,19,198,204,204,204}', '[......#.] (0,3) (0,1,2,4,5,6) (0,3,4,6) (1,2,4) (0,1,3,4,6) (0,1,2) (1,2,3,5) (2,5,7) (0,2) (0,1,5) {100,82,74,65,58,68,52,13}', '[#.##...] (3,6) (2,4,5) (3) (1,2,3) (0,1,2,3) (1,4,5) (0,2,6) {20,30,39,30,20,20,16}']
R=168, C=33
16463
